# Test of OpenCV + FastAI for video capture

In [37]:
# !pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

from fastbook import *
from fastai.vision.widgets import *
import fastai

import numpy as np
import cv2

<br>
<br>

## Load the model

In [21]:
path = Path('../models')
learn_inf = load_learner(path/'export-resnet18-tune4.pkl')

<br>
<br>

## Open the video camera, loop for every Frame, massage the image and make a prediction.  

In [35]:


# cap = cv2.VideoCapture('http://127.0.0.1:4747/video')
cap = cv2.VideoCapture('http://10.0.0.75:4747/video')

nframe = 0  # number of frames

# "Game" Loop ... for every frame
while(True):
    nframe+=1
    # Capture frame-by-frame
    ret, frame = cap.read()

    # only process every 5 frames (~6 frames/sec)
    if nframe % 5 == 0:
        # convert the frame to a PILimage that fastai can read
        # pframe = fastai.vision.core.PILImage(to_image(frame))

        # resize the image to 128x128 and rotate it
        # @TODO: accomplish this while reading the camera
        frame = cv2.resize(frame, (128,128))
        frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
        
        # create the prediction
        pred,pred_idx,probs = learn_inf.predict(frame)
        
        # for now just print it to the console (only if it's a sufficiently height guess)
        if probs[pred_idx] > 0.6:
            print(f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}')

        
        # Display the resulting frame
        cv2.imshow('frame',frame)
   
    # Break out of the game loop if the user presses 'Q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print(f'Frame shape = {frame.shape}')
        print(f'Number of Frames = {nframe}')
        break


# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

Prediction: F; Probability: 0.6256


Prediction: H; Probability: 0.7049


Prediction: G; Probability: 0.6760


Prediction: G; Probability: 0.6760


Prediction: A; Probability: 0.7935


...

Prediction: I; Probability: 0.9650


Prediction: I; Probability: 0.9774


Prediction: I; Probability: 0.9380


Prediction: H; Probability: 0.8713


Frame shape = (480, 640, 3)
Number of Frames = 211


<br>
<br>
<br>
<br>
<br>


## The rest of this file is for reference only.  The code here is more complete than my prototype and I'll want to add it back later.

In [1]:
import cv2
import numpy as np

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture('Chaplin.mp4')

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:

    # Display the resulting frame
    cv2.imshow('Frame',frame)

    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

